# Thành viên nhóm:
- Nguyễn Thị Tình - 1612703
- Trang Trung Hoàng Phúc - 1612521

In [1]:
import urllib.robotparser
rp = urllib.robotparser.RobotFileParser()
rp.set_url('https://freesound.org/')
rp.read()
rp.can_fetch('*', 'https://freesound.org/')

True

In [19]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from tqdm import tqdm_notebook as tqdm
import glob

In [7]:
driver = webdriver.Chrome(executable_path=r'chromedriver.exe')
driver.maximize_window()

In [8]:
# Login
driver.get('https://freesound.org/home/login/?next=/')
username = driver.find_element_by_id('id_username')
username.send_keys('sounddetection')
password = driver.find_element_by_id('id_password')
password.send_keys('12345')
password.send_keys(Keys.RETURN)

In [12]:
# Get all links
keywords = ['dog bark', 'baby cry', 'say go']
for keyword in keywords:
    audio_href = []
    search_box = driver.find_element_by_id('search')
    search_box = search_box.find_element_by_tag_name('input')
    search_box.send_keys(keyword)
    search_box.send_keys(Keys.RETURN)
    # Only get first 5 pages
    with open(keyword + '.txt', 'w') as f:
        for _ in range(5):
            audio_urls = driver.find_elements_by_class_name('title')
            for url in audio_urls:
                f.write(url.get_attribute('href') + '\n')
            next_page = driver.find_element_by_class_name('next-page')
            next_page = next_page.find_element_by_tag_name('a')
            next_page.click()
    

In [20]:
def enable_download_headless(browser,download_dir):
    browser.command_executor._commands["send_command"] = ("POST", '/session/$sessionId/chromium/send_command')
    params = {'cmd':'Page.setDownloadBehavior', 'params': {'behavior': 'allow', 'downloadPath': download_dir}}
    browser.execute("send_command", params)
# Download an audio base on an link
def download_file(url):
    driver.get(url)
    download_button = driver.find_element_by_id('download_button')
    download_button.click()
    
def wait_for_downloads(folder):
    unfinished_files = glob.glob(folder + "\\*.crdownload")
    if len(unfinished_files) >= 5:
        return True
    else:
        return False

In [21]:
for keyword in keywords:
    download_dir = 'D:\Learning\DataScience\Project\DataScienceFinalProject\FreesoundAudio\\' + keyword
    enable_download_headless(driver, download_dir)
    with open(keyword + '.txt', 'r') as f:
        for url in f.readlines():
            download_file(url)
            while wait_for_downloads(download_dir):
                time.sleep(5)